In [ ]:
import pandas as pd
from utils_24 import p1_place
pd.read_csv('Wimbledon_featured_matches.csv')

In [ ]:
df = pd.read_csv('Wimbledon_featured_matches.csv').ffill()
df[['p1_ace','p2_ace','p1_winner','p2_winner','p1_break_pt_won','p2_break_pt_won','p1_break_pt_missed','p2_break_pt_missed','p1_net_pt_won','p2_net_pt_won','p1_double_fault','p2_double_fault','p1_unf_err','p2_unf_err']] = df[['p1_ace','p2_ace','p1_winner','p2_winner','p1_break_pt_won','p2_break_pt_won','p1_break_pt_missed','p2_break_pt_missed','p1_net_pt_won','p2_net_pt_won','p1_double_fault','p2_double_fault','p1_unf_err','p2_unf_err']].shift().fillna(0)

In [ ]:
df

In [ ]:
df['elapsed_time'] = pd.to_timedelta(df['elapsed_time']).apply(lambda x: x.total_seconds()/60)
df.elapsed_time = df.elapsed_time.diff().fillna(0).apply(lambda x: max(0, x))
df.rename(columns={'elapsed_time': 'last_time'}, inplace=True) 

In [ ]:
df = p1_place(df)
df = pd.get_dummies(df, columns=['serve_width','serve_depth','return_depth'],drop_first=True) #p1_place,这个在上面编码了
df[['serve_width_BC', 'serve_width_BW', 'serve_width_C', 'serve_width_W', 'serve_depth_NCTL',
'return_depth_ND']] = df[['serve_width_BC', 'serve_width_BW', 'serve_width_C', 'serve_width_W',
                          'serve_depth_NCTL','return_depth_ND']].astype(int)

grouped = df.groupby('match_id')
grouped_dfs = {}
for name, group_df in grouped:
    grouped_dfs[name] = group_df

In [ ]:
df.columns

In [ ]:
select_columns = ['last_time','set_no','game_no',
                  'server','p1_place',
                  'p1_sets','p2_sets','p1_games','p2_games','p1_points_won','p2_points_won',
                  'p1_break_pt','p2_break_pt']
df = df[df['player1']=='Carlos Alcaraz']
train = df[df['player2'] != 'Nicolas Jarry']
test = df[df['player2'] == 'Nicolas Jarry']
# train = df
# test = df

X_train = train.drop(['point_victor','player1','player2','match_id','p1_score','p2_score','winner_shot_type'],axis=1)
X_test = test.drop(['point_victor','player1','player2','match_id','p1_score','p2_score','winner_shot_type'],axis=1)

# X_train = train[select_columns]
# X_test = test[select_columns]

y_train = train['point_victor'].replace(2,0)
y_test = test['point_victor'].replace(2,0)

In [ ]:
X_train.info()


X_train.to_csv('X_train_2.csv',index=False)
y_train.to_csv('y_train_2.csv',index=False)
X_test.to_csv('X_test_2.csv',index=False)
y_test.to_csv('y_test_2.csv',index=False)